<a href="https://colab.research.google.com/github/akrm3008/deep-gasoline/blob/master/testing_LLMS_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install evaluate

In [ ]:
import transformers
import huggingface_hub
from huggingface_hub import notebook_login
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import evaluate
from datasets import Dataset, load_dataset
from peft import LoraConfig, get_peft_model, TaskType
from transformers import DataCollatorWithPadding

In [ ]:
notebook_login()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/tweet_text_2.csv", encoding = 'cp1252')
data = data[data.Shortage.notna()]
train_data = data.sample(frac =0.8)
test_data = data[~data.index.isin(train_data.index)]


In [ ]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)


class Model(nn.Module):

    def __init__(self, num_labels):
        super().__init__()
        self.bert = model
        self.linear = nn.Linear(768,num_labels)
        self.softmax = nn.Softmax(1)
    def forward(self,token_ids, attention_masks ):
        _,bert_output = model(token_ids, attention_mask=attention_masks,return_dict=False)
        output = self.softmax(self.linear(bert_output))
        return output

num_labels = 2
finetune_model = Model(num_labels)
finetune_model.to(device)

Model(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=

In [ ]:
for params in finetune_model.parameters():
    params.requires_grad = False

for params in finetune_model.linear.parameters():
    params.requires_grad = True


In [ ]:
batch_size = 32
criterion = nn.CrossEntropyLoss()
learning_rate = 1e-5
optimizer = torch.optim.Adam(finetune_model.parameters(), lr=learning_rate)

In [ ]:
class CustomDataSet(Dataset):
    def __init__(self, token_ids, attention_masks, labels):
        self.tokens = token_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.tokens[idx], self.attention_masks[idx], self.labels[idx]

train_sample = tokenizer.batch_encode_plus(list(train_data["TWEET_TEXT"]), padding = "max_length", truncation=True)
test_sample = tokenizer.batch_encode_plus(list(test_data["TWEET_TEXT"]), padding = "max_length", truncation=True)
train_dataset = CustomDataSet(torch.tensor(train_sample["input_ids"]), torch.tensor(train_sample["attention_mask"]), torch.tensor(list(train_data['Shortage'])))
test_dataset = CustomDataSet(torch.tensor(test_sample["input_ids"]), torch.tensor(test_sample["attention_mask"]), torch.tensor(list(test_data['Shortage'])))
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size)


In [ ]:
device

device(type='cuda')

In [ ]:
loss_adam = []
n_epochs = 10


for epoch in range(n_epochs):
    epoch_loss_train, epoch_loss_test, epoch_accuracy_train, epoch_acccuracy_test = 0, 0, 0, 0
    outputs_train, outputs_test, labels_epochs_train, labels_epochs_test  = [], [], [], []
    n = 0
    for step, batch in enumerate(train_dataloader):
        n += 1
        batch_loss = 0
        input_ids, attention_masks, labels = batch
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        labels = labels.to(device)
        output = finetune_model(input_ids, attention_masks)
        loss = criterion(output, labels.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print("here")
        print(f"epoch/total epoch : {epoch}/{n_epochs}---- step:{step}----- loss:{loss.item()}")
        epoch_loss_train += loss.item()
        print(output)
        print('------------------')
        print(labels)
        output = output.detach().cpu().numpy()
        labels = labels.detach().cpu().numpy()
        print('xxxxxxxx')
        print(output)
        print('------------------')
        print(labels)
        outputs_train.append(output)
        labels_epochs_train.append(labels)
    #outputs_train = np.concatenate(outputs_train, axis =None)
    #labels_epochs_train = np.concatenate(labels_epochs_train, axis =None)
    #epoch_accuracy_train = sum(outputs_train == labels_epochs_train)/(outputs_train.shape[0])
 #   for step, batch in enumerate(test_dataloader):
 #       input_ids, attention_masks, labels = batch
 #       input_ids = input_ids.to(device)
 #       attention_masks = attention_masks.to(device)
 #       labels = labels.to(device)
 #       output = finetune_model(input_ids, attention_masks)
 #       loss = criterion(output, labels.long())
 #       epoch_loss_test += loss.item()
 #       output = output.detach().cpu().numpy()
 #       labels = labels.detach().cpu().numpy()
 #       outputs_test.append(output)
 #       labels_epochs_test.append(labels)
    #outputs_test = np.concatenate(outputs_test, axis =None)
    #labels_epochs_test = np.concatenate(labels_epochs_test, axis =None)
    #epoch_accuracy_test = sum(outputs_test == labels_epochs_test)/(outputs_test.shape[0])
    #print(f"epoch/total epoch : {epoch}/{n_epochs} --- train loss:{epoch_loss_train/n} --- test loss:{epoch_loss_test/n} --- train accuracy:{epoch_accuracy_train} --- test accuracy:{epoch_accuracy_test}")




KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Finetuning all layers

In [ ]:
train_data["Shortage"] = train_data["Shortage"].astype(int)
test_data["Shortage"] = test_data["Shortage"].astype(int)

finetune_model_2 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
training_args = TrainingArguments(output_dir="/content/drive/MyDrive/Irma_Tweets_Modeling/BERT_training/All_layers_finetuned",
                                  eval_strategy="epoch", num_train_epochs=5, save_strategy="epoch", load_best_model_at_end=True)
metrics = evaluate.combine(['accuracy', 'precision', 'recall', 'f1'])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis =-1)
    return metrics.compute(predictions=predictions, references=labels)

train_dataset = Dataset.from_pandas(train_data, split='train')
test_dataset = Dataset.from_pandas(test_data, split='test')

def tokenize_function(examples):
    return tokenizer(examples["TWEET_TEXT"], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
train_dataset = train_dataset.rename_column("Shortage", "labels")
test_dataset = test_dataset.rename_column("Shortage", "labels")
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])


trainer = Trainer(
    model=finetune_model_2,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


<ipython-input-16-bd91290b471d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["Shortage"] = test_data["Shortage"].astype(int)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2799 [00:00<?, ? examples/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.455384,0.834286,0.850187,0.749175,0.796491
2,0.458800,0.434622,0.847143,0.824503,0.821782,0.823140
3,0.282900,0.602591,0.847143,0.840278,0.798680,0.818951
4,0.282900,0.794847,0.848571,0.804954,0.858086,0.830671
5,0.104600,0.898616,0.847143,0.804348,0.854785,0.828800


TrainOutput(global_step=1750, training_loss=0.24574002756391253, metrics={'train_runtime': 371.3161, 'train_samples_per_second': 37.69, 'train_steps_per_second': 4.713, 'total_flos': 3682239219763200.0, 'train_loss': 0.24574002756391253, 'epoch': 5.0})

Finetuning only the last layer


In [ ]:
finetune_model_3 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

for params in finetune_model_3.parameters():
    params.requires_grad = False

for params in finetune_model_3.classifier.parameters():
    params.requires_grad = True

training_args = TrainingArguments(output_dir="/content/drive/MyDrive/Irma_Tweets_Modeling/BERT_training/final_layer_finetuned",
                                  eval_strategy="epoch", num_train_epochs=10, save_strategy="epoch", load_best_model_at_end=True)


trainer = Trainer(
    model=finetune_model_3,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.675862,0.572857,0.534483,0.102310,0.171745
2,0.683900,0.672543,0.605714,0.545763,0.531353,0.538462
3,0.675300,0.670182,0.570000,1.000000,0.006601,0.013115
4,0.675300,0.663119,0.585714,0.740741,0.066007,0.121212
5,0.676400,0.660088,0.614286,0.624060,0.273927,0.380734
6,0.671500,0.658748,0.600000,0.671642,0.148515,0.243243
7,0.671500,0.657669,0.600000,0.671642,0.148515,0.243243
8,0.674200,0.656245,0.620000,0.643411,0.273927,0.384259
9,0.664700,0.655763,0.618571,0.632353,0.283828,0.391800
10,0.669200,0.655659,0.620000,0.635036,0.287129,0.395455


TrainOutput(global_step=3500, training_loss=0.6736045706612723, metrics={'train_runtime': 271.3855, 'train_samples_per_second': 103.137, 'train_steps_per_second': 12.897, 'total_flos': 7364478439526400.0, 'train_loss': 0.6736045706612723, 'epoch': 10.0})

testing LLama

In [ ]:
model_name = "meta-llama/Meta-Llama-3-8B"
finetune_model_4 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
#finetune_model_4.resize_token_embeddings(len(tokenizer))
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})
finetune_model_4.config.pad_token_id = finetune_model_4.config.eos_token_id
finetune_model_4.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(128256, 4096)

In [ ]:
metrics = evaluate.combine(['accuracy', 'precision', 'recall', 'f1'])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis =-1)
    return metrics.compute(predictions=predictions, references=labels)

In [ ]:

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

def tokenize_function(examples):
    return tokenizer(examples["TWEET_TEXT"], truncation=True)

train_data["Shortage"] = train_data["Shortage"].astype(int)
test_data["Shortage"] = test_data["Shortage"].astype(int)
train_dataset = Dataset.from_pandas(train_data, split='train')
test_dataset = Dataset.from_pandas(test_data, split='test')
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
train_dataset = train_dataset.rename_column("Shortage", "labels")
test_dataset = test_dataset.rename_column("Shortage", "labels")
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])




<ipython-input-7-81ad63bc6e86>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["Shortage"] = test_data["Shortage"].astype(int)


Map:   0%|          | 0/2799 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/700 [00:00<?, ? examples/s]

int

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1
)
finetune_model_peft_4 = get_peft_model(finetune_model_4, peft_config)
finetune_model_peft_4.resize_token_embeddings(len(tokenizer))

Embedding(128256, 4096)

In [ ]:
training_args = TrainingArguments(output_dir="/content/drive/MyDrive/Irma_Tweets_Modeling/Llama_peft_training/",
                                  eval_strategy="epoch", num_train_epochs=10, save_strategy="no")

trainer = Trainer(
    model=finetune_model_peft_4,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

<ipython-input-9-2be43d994d83>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: abhinavkhr30 (abhinavkhr30-amazon). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.647697,0.752857,0.742537,0.656766,0.697023
2,0.685700,0.506351,0.832857,0.839416,0.759076,0.797227
3,0.335300,0.552621,0.844286,0.814935,0.828383,0.821604
4,0.335300,0.690158,0.852857,0.824675,0.838284,0.831424
5,0.167300,0.774594,0.847143,0.860294,0.772277,0.813913
6,0.053300,0.832397,0.861429,0.862676,0.808581,0.834753
7,0.053300,0.941654,0.868571,0.837061,0.864686,0.850649
8,0.013600,0.960209,0.868571,0.857627,0.834983,0.846154
9,0.000800,0.984802,0.871429,0.851485,0.851485,0.851485
10,0.000900,0.986907,0.871429,0.851485,0.851485,0.851485


TrainOutput(global_step=3500, training_loss=0.1795694570030485, metrics={'train_runtime': 2404.5057, 'train_samples_per_second': 11.641, 'train_steps_per_second': 1.456, 'total_flos': 4.622765349843763e+16, 'train_loss': 0.1795694570030485, 'epoch': 10.0})

In [ ]:
finetune_model_peft_4.save_pretrained("/content/drive/MyDrive/Irma_Tweets_Modeling/LLamma_training/LLama_peft_training/final_model")


In [ ]:
tokenizer.save_pretrained("/content/drive/MyDrive/Irma_Tweets_Modeling/LLamma_training/LLama_peft_training/final_model")

('/content/drive/MyDrive/Irma_Tweets_Modeling/LLamma_training/LLama_peft_training/final_model/tokenizer_config.json',
 '/content/drive/MyDrive/Irma_Tweets_Modeling/LLamma_training/LLama_peft_training/final_model/special_tokens_map.json',
 '/content/drive/MyDrive/Irma_Tweets_Modeling/LLamma_training/LLama_peft_training/final_model/tokenizer.json')